In [ ]:
from trustutils import run
import matplotlib.pyplot as plt
import numpy 

run.introduction("Valentin Kraemer - 13/11/2025")

# Robin boundary conditions for Stokes and Navier-Stokes equation with VEF discretization

## Mathematical description

We introduce here a new boundary condition for VEF discretization: the Robin boundary conditions for Stokes and Navier-Stokes equations. 
For more information about VEF formulation, please refer to this article [VEF description](https://doi.org/10.1002/fld.5399). 


Robin boundary conditions consist in a linear combination between the flux term $\boldsymbol{F}$ and the variables term. This type of boundary conditions can be useful for Fluid structures interactions or domain decomposition for instance. 
The implementation in TRUST decomposes between the normal and the tangential part. We define the outward normal vector $\boldsymbol{n} = (n_x, n_y)^t$. 

The flux term becomes:

\begin{equation}
    \boldsymbol{F} = F_n \boldsymbol{n} + \boldsymbol{F_t}
\end{equation}
    
with $F_n$ the normal part of the flux term and $\boldsymbol{F_t}$ the tangential part. 

For the Navier-Stokes equations, the flux term can be written as follows:



$$  \begin{aligned}
    F_n &= \nu \nabla_n \boldsymbol{u} \cdot \boldsymbol{n} + \chi (\boldsymbol{u}\cdot \boldsymbol{n})(\boldsymbol{u}\cdot  \boldsymbol{n}) - p \\
    \boldsymbol{F_t} &= \nu \nabla_n \boldsymbol{u} \times \boldsymbol{n} + \chi (\boldsymbol{u}\cdot \boldsymbol{n})(\boldsymbol{u}\times \boldsymbol{n})
\end{aligned}
$$

with $\nu$ the viscosity, and $\chi\in \{0,1\}$ (0 for Stokes and 1 for NS). 
In 2D, we project on the tangential vector $\boldsymbol{t} = (-n_y, n_x)$ instead of applying the cross product $\times \boldsymbol{n}$ (the tangential part consists of only one vector in 2D).

Then, let's define the Robin parameters $\alpha$ for the normal part and $\beta$ for the tangential part and the Robin data:

- a normal data $g_N$ which is a scalar function,
- a tangential data $\boldsymbol{g_T}$ which is a scalar function in 2D and a vectorial function in 3D.  

    




The Robin boundary conditions which have been implemented in TRUST are defined such that: 


$$  \begin{aligned}
    \alpha F_n + \boldsymbol{u}\cdot \boldsymbol{n} &= g_N\\
    \beta \boldsymbol{F_t}  + \boldsymbol{u}\times \boldsymbol{n} &=\boldsymbol{g_T}
\end{aligned}
$$

In TRUST, we use the keyword ``Robin_VEF`` for the Robin boundary conditions, with the following parameters: 

- ``alpha``, ``beta``, defined bellow. 
- the keyword ``champ_front_normal_et_tangentiel``, followed by the associated field data  (consider that the following field as a concatenation with $g_N$ and $\boldsymbol{n}\times \boldsymbol{g_T}$ ) 

There is a 2D example of a Navier-Stokes problem for $\boldsymbol{u}=(y,-x)^t$, $p=0.5(x^2+y^2)-1/3$, and $\boldsymbol{n}=(1,0)^t$.


    Robin_VEF {
            alpha 3
            beta 4
            champ_front_normal_et_tangentiel_robin champ_front_fonc_txyz 2 -1.5x^2-4.5y^2+y+1.0 4xy-x-4         
    }

In this example, the first function in the field corresponds to $g_N$ and the second is $g_T$. 


In 3D, the field ``champ_front_normal_et_tangentiel`` will have 4 components (one for $g_N$ and three for $\boldsymbol{n}\times\boldsymbol{g_T}$). 

Note that we use $\boldsymbol{n}\times\boldsymbol{g_T}$ because we want to write the real tangential component of $\boldsymbol{u}$.

## Test cases  

The main part of the test cases presented in this file comes from the [FVCA8 Benchmark](https://hal.science/hal-04019649), which has already been studied with Dirichlet boundary conditions by the TRUST-TrioCFD team [in this paper](https://cea.hal.science/cea-02434556). 

The other test cases come from the internship of K. Ait-Ameur for a non orthogonal domain and from a TrioCFD technical note. 
.



### 2D Test cases 
- Steady Bercovier-Englemann

$$  
\begin{aligned}
    u &= 
    \begin{pmatrix}
        -256 x^2 y (1-x)^2  (y-1) (2 y-1)\\
        ~~256 y^2 x (1-y)^2 (x-1) (2 x-1) 
    \end{pmatrix}
    \\
    p &= (x-0.5) (y-0.5)
 \end{aligned} $$


- Steady Taylor-Green vortex

$$  \begin{aligned}
    u &= 
    \begin{pmatrix}
        -\sin(2  \pi  y) \cos(2  \pi  x) \\
        ~ \sin(2  \pi  x) \cos(2  \pi  y)
    \end{pmatrix}
    \\
    p &= -\frac{1}{4} (\cos(4  \pi  x)+ \cos(4  \pi  y))
 \end{aligned} $$

 
- Unsteady Taylor-Green vortex (with t polynom)

$$  \begin{aligned}
    u &= t 
    \begin{pmatrix}
    - \sin(2  \pi  y) \cos(2  \pi  x)\\
    ~ \sin(2  \pi  x) \cos(2  \pi  y)\\
    \end{pmatrix}
    \\
    p &= -\frac{t}{4} (\cos(4  \pi  x) +\cos(4  \pi  y)) 
 \end{aligned} $$
 
- Steady Navier-Stokes

$$  \begin{aligned}
    u &= 
    \begin{pmatrix}
        ~y\\
        -x
    \end{pmatrix}
    \\
    p &= \frac{1}{2} (x^2+y^2)-\frac{1}{3}
 \end{aligned} $$
  


- Unsteady Navier-Stokes

$$  \begin{aligned}
    u &= e^{-5  \pi^2  t}
    \begin{pmatrix}
            -2  \pi  \cos( \pi  x) \sin(2  \pi  y)  \\
            \pi  \sin( \pi  x) \cos(2  \pi  y) 
    \end{pmatrix}
    \\
    p &= -\frac{\pi^2} {4} e^{-10  \pi^2  t} (4 cos(2  \pi  x)+ cos(4  \pi  y)) 
 \end{aligned} $$
  


- Katia Ait Ameur test case

We use a particular geometry for this test case, presented as followed ![picture](ameur_geo.png). 

The Robin boundary conditions are imposed at the top of the domain, and the velocity pressure couple used in this test-case is:

$$  \begin{aligned}
    u &=
    \begin{pmatrix}
    \cos( \pi  t) \sin( \pi  y)\\
    \cos( \pi  t) \sin( \pi  x)) 
    \end{pmatrix}
    \\
    p &= -\frac{1}{2} \cos( \pi  x) \cos( \pi  t)
 \end{aligned} $$



### 3D Test cases 
- Steady Taylor-Green vortex

$$  \begin{aligned}
    u &= 
    \begin{pmatrix}
        -2 \cos(2  \pi  x) \sin(2  \pi  y) \sin(2  \pi  z) \\
        ~~~~~\sin(2  \pi  x) \cos(2  \pi  y) \sin(2  \pi  z) \\
        ~~~~~\sin(2  \pi  x) \sin(2  \pi  y) \cos(2  \pi  z))
    \end{pmatrix}
    \\
    p &= -6  \pi  \sin(2  \pi  x) \sin(2  \pi  y) \sin(2  \pi  z)
 \end{aligned} $$
  
- Unsteady Taylor-Green vortex (with t polynom)

$$  \begin{aligned}
    u &= t
   \begin{pmatrix}
    -2 \cos(2  \pi  x) \sin(2  \pi  y) \sin(2  \pi  z)  \\
     ~~~~~ \sin(2  \pi  x) \cos(2  \pi  y) \sin(2  \pi  z)  \\
     ~~~~~ \sin(2  \pi  x) \sin(2  \pi  y) \cos(2  \pi  z) 
    \end{pmatrix}
    \\
    p &= -6  \pi  \sin(2  \pi  x) \sin(2  \pi  y) \sin(2  \pi  z) t
 \end{aligned} $$

- Steady Navier-Stokes

$$  \begin{aligned}
    u &= 
    \begin{pmatrix}
        y-z \\
        z-x \\ 
        x-y
    \end{pmatrix}    
    \\
    p &= x^2+y^2+z^2-x y-x z-y z-1/4 
 \end{aligned} $$

- Unsteady Navier Stokes

$$  \begin{aligned}
    u &= (t^2+1)
    \begin{pmatrix}
        y-z \\
        z-x \\ 
        x-y
    \end{pmatrix}    
    \\
    p &= (t^2+1)(x^2+y^2+z^2-x y-x z-y z-1/4)
 \end{aligned} $$

Work in progress with Generalized Beltrami flow

 ### Parameters

In [ ]:
nb_meshes_max = 2 # max 7 for 2D meshes
meshsize = 0.12
hlist = [meshsize/(2**i) for i in range(0,nb_meshes_max) ]
h2list = list(numpy.array(hlist)**2)
dt = h2list 

# Run Test cases

In [ ]:
import os, sys
from string import Template
seq = True

#mesh_name, discretization, number_of_mesh_max#
m2d = {
    # 2D meshes
    "Triangles":      (["VEFP0"], 7),
}
m3d = {
    # 3D meshes
    "Tetra":          (["VEFP0"], 5),
}

mxd = { 2 : m2d, 3 : m3d }
tests = {
    "steady_Bercovier_Englemann" : 2,
    "steady_Taylor_Green2D" : 2, 
    "unsteady_Taylor_Green2D" : 2,
    "steady_NS2D" : 2, 
    "unsteady_NS2D" : 2,
    "ameur_tc" : 2, 
    "steady_Taylor_Green3D" : 3,
    "steady_NS3D" : 3, 
    "unsteady_Taylor_Green3D" : 3,
    "unsteady_NS3D" : 3, 
}

tests2d = {
    "steady_Bercovier_Englemann" : 2,
    "steady_Taylor_Green2D" : 2, 
    "unsteady_Taylor_Green2D" : 2,
    "steady_NS2D" : 2, 
    "unsteady_NS2D" : 2,
    "ameur_tc" : 2, 
}

tests3d = {
    "steady_NS3D" : 3, 
    "unsteady_Taylor_Green3D" : 3,
    "unsteady_NS3D" : 3, 
}



In [ ]:

run.reset()

run.initBuildDirectory()
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
list_dis = []
for test, dim in tests.items():
    for m, (dis, ns) in mxd[dim].items():                   # loop on mesh_name m with its items (dis discretization and number of mesh max)
        for d in dis :                                      # loop on discretization 
            for n in range(1, min(ns, nb_meshes_max) + 1):  # loop on mesh size
                # rafiner isotrope 
                raf = "".join(["Raffiner_isotrope dom "  for _ in range(n-1)])              
                
                # folder creation 
                os.system(f"mkdir -p {test}/{m}/{d}/{n}")

                # opening test case
                with open(f"{test}.data", "r") as file: filedata = Template(file.read())

                # change discretization 
                if d.startswith("VEFP0") : 
                    result = filedata.substitute({
                        "dis": "VEFPreP1b dis Read dis { P0 }",
                        "raf": raf, 
                        "dt": dt[n-1]
                    })                
                else : 
                    result = filedata.substitute({"dis" : d +" dis"})

                # writing jdd 
                with open(f"{test}/{m}/{d}/{n}/jdd.data", "w") as file: file.write(result)
                  
                np = 1 if seq else min(int(os.environ["TRUST_NB_PROCS"]), 8 if n < 2 else 128)
                os.system(f"cp post_run {test}/{m}/{d}/{n}/")
                if not seq: os.system(f"cd {test}/{m}/{d}/{n}/; make_PAR.data jdd {np} >/dev/null 2>&1")
                #echo $mesh/jdd_${n}/cas.data >> ll
                cas = run.addCase(f"{test}/{m}/{d}/{n}", f"{'' if seq else 'PAR_'}jdd.data", np)


                #cas.substitute("_lance_test_","")
                list_dis.append(f"{test}/{m}/{d}")
                
with open("extract_convergence", "r") as file: fileconv = Template(file.read())
result = fileconv.safe_substitute(list_dis=" ".join(sorted(list(set(list(list_dis))))))
with open("extract_convergence", "w") as file: file.write(result)
os.chdir(origin)

run.printCases()
run.runCases()
run.executeScript("extract_convergence")



# Convergence rates

In [ ]:
dim = 2
conv_glob_unk = ["conv_glob_p", "conv_glob_v"] # + "conv_glob_divu"
colors = ["g", "r"]
labels = ["LinfL2 Pressure error", "LinfL2 Velocity error"]

for test in tests2d : 
    for m, (dis, ns) in mxd[dim].items():
        for d in dis : 
            for i, conv in enumerate(conv_glob_unk) :  
                conv_glob = f"build/{test}/{m}/{d}/{conv}"
                data = numpy.loadtxt(conv_glob)
                print(data)
                h = data[:, 0]
                err = data[:, 1]
                plt.loglog(h, err, color = colors[i], label =labels[i], linestyle ="-", marker = "o")
                plt.loglog(h, h, color = "k", label ="order 1", linestyle ="-", marker = "o")
                plt.loglog(h, h*h, color = "k", label ="order 2", linestyle =":", marker = "d")
                err = numpy.flip(err)
                h = numpy.flip(h)
                order = numpy.log(err[:-1]/err[1:])/ numpy.log(h[:-1]/h[1:])
                print(f"order for {test}/{conv} is {order} ")
            plt.xlabel("h")
            plt.ylabel("error")
            plt.grid(True, which="both", ls="-")
            plt.legend()
            plt.title(f"Error for {test}")
            plt.show()
            

In [ ]:
dim = 3
conv_glob_unk = ["conv_glob_p", "conv_glob_v"] # + "conv_glob_divu"
colors = ["g", "r"]
labels = ["LinfL2 Pressure error", "LinfL2 Velocity error"]

for test in tests3d : 
    for m, (dis, ns) in mxd[dim].items():
        for d in dis : 
            for i, conv in enumerate(conv_glob_unk) :  
                conv_glob = f"build/{test}/{m}/{d}/{conv}"
                data = numpy.loadtxt(conv_glob)
                h = data[:, 0]
                err = data[:, 1]
                plt.loglog(h, err, color = colors[i], label =labels[i], linestyle ="-", marker = "o")
                plt.loglog(h, h, color = "k", label ="order 1", linestyle ="-", marker = "o")
                plt.loglog(h, h*h, color = "k", label ="order 2", linestyle =":", marker = "d")
                err = numpy.flip(err)
                h = numpy.flip(h)
                order = numpy.log(err[:-1]/err[1:])/ numpy.log(h[:-1]/h[1:])
                print(f"order for {test}/{conv} is {order} ")
            plt.xlabel("h")
            plt.ylabel("error")
            plt.grid(True, which="both", ls="-")
            plt.legend()
            plt.title(f"Error for {test}")
            plt.show()
            